In [1]:
import cv2
import time
import ipywidgets
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from apocd import apocd_process_images
from jetcam.csi_camera import CSICamera



left_camera = CSICamera(capture_device=0, width=224, height=224, capture_fps=30)
right_camera = CSICamera(capture_device=1, width=224, height=224, capture_fps=30)

frame0 = left_camera.read()
frame1 = right_camera.read()

left_camera.running = True
right_camera.running = True

In [2]:
distance_slider_widget = widgets.FloatSlider(description='Distance', min=0, max=100, value=0)

image_widget = widgets.Image()
traitlets.dlink((left_camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

image_box = widgets.HBox([image_widget])

widgets_box = widgets.VBox([image_box, distance_slider_widget])

In [3]:
from jetbot import Robot
robot = Robot()

In [4]:

frame0 = None
frame1 = None


def update_image0(change):
    global frame0
    frame0 = change['new']
#     print("frame0 updated")

def update_image1(change):
    global frame1
    frame1 = change['new']
#     print("frame1 updated")
    
left_camera.observe(update_image0, names='value')
right_camera.observe(update_image1, names='value')


def process_and_control():
    global frame0, frame1

    try:
        while True:
            if frame0 is not None and frame1 is not None:
                status, coords, distance, dis_color, disp = apocd_process_images(frame0, frame1)

                if not isinstance(distance, float):
                    distance = 0
                
                distance_slider_widget.value = distance
#                 print(status)
                
                if distance < 0.30:
                    robot.stop()
                else:
                    robot.forward(speed=0.3)
                
                if coords is not None:
                    X, Y, Z = coords
                    print(f"World Coordinates: X = {X} mm, Y = {Y} mm, Z = {Z} mm")
                    print(f"Distance: {distance} m")
            else:
                print("Unable to read input images")
            
            time.sleep(0.03)  # Control frame rate
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except KeyboardInterrupt:
        pass
    finally:
        robot.stop()


In [ ]:
display(widgets_box)
process_and_control()